# Image Classification

## Case Study
Scrap images of jeans, trousers and sarees from Amazon to train and test the image classification model using CNN model.

In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

In [31]:
PATH = "/Users/bhupendrakumar/Desktop/chromedriver"
driver = webdriver.Chrome(PATH)

In [34]:
page=1
iTotImages=0
#Scrapping 300 images for each cloth
while(iTotImages<300):
    #get next page
    driver.get("https://www.amazon.in/s?k=jeans&page=2&qid=1623357882&ref=sr_pg_"+str(page))
    images=driver.find_elements_by_xpath('//img[@class="s-image"]')
    
    for iIter in range(len(images)):
        src=images[iIter].get_attribute('src')
        response=requests.get(src)
        file = open("/Users/bhupendrakumar/Documents/Personal/Internship/Jeans/img"+str(iTotImages)+".jpg", "wb")
        file.write(response.content)
        iTotImages+=1
    page+=1

# Scratching train data set

In [63]:
# Importing libraries related to pytorch
import torch
import glob
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib

In [57]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [61]:
transformer=transforms.Compose([transforms.Resize((150,150)),transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])

In [66]:
#train/test directory
folder_path="/Users/bhupendrakumar/Documents/Personal/Internship/Image_Data"
test_folder_path="/Users/bhupendrakumar/Documents/Personal/Internship/Test Data"

#Dataloader
train_dataloader=DataLoader(torchvision.datasets.ImageFolder(folder_path,transform=transformer),
                           batch_size=100,shuffle=True)

test_dataloader=DataLoader(torchvision.datasets.ImageFolder(test_folder_path,transform=transformer),
                           batch_size=100,shuffle=True)

In [78]:
# Now we will get the convolutional NN network class
root=pathlib.Path(folder_path)
classes=[j.name.split('/')[-1] for j in root.iterdir()]

In [79]:
classes=classes[1:]

In [80]:
classes

['Jeans', 'Sarees', 'Trousers']

In [81]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
        #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [82]:
model=ConvNet(num_classes=3).to(device)

In [83]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [84]:
num_epochs=10

In [85]:
#calculating the size of training and testing images
train_count=len(glob.glob(folder_path+'/**/*.jpg'))
test_count=len(glob.glob(test_folder_path+'/**/*.jpg'))

In [96]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_dataloader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_dataloader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(1.3099e-05) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 1 Train Loss: tensor(0.0001) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 2 Train Loss: tensor(0.0001) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 3 Train Loss: tensor(1.1925e-06) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 4 Train Loss: tensor(2.7395e-06) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 5 Train Loss: tensor(5.5537e-07) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 6 Train Loss: tensor(1.3224e-06) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 7 Train Loss: tensor(1.0489e-06) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 8 Train Loss: tensor(1.0777e-06) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 9 Train Loss: tensor(5.9401e-07) Train Accuracy: 1.0 Test Accuracy: 1.0


In [ ]:
# Build CNN model above is giving us accuracy 1. We can use this model ahead for unseen images predictions.